## Contour deformation 

In the context of GW method, contour deformation (CD) technique is used in conjunction with resolution of identity (RI) to reduce the formal scaling of the self-energy calculation. Compared to widely used analytic continuation approach it provides a means to evaluate self-energy  directly on the real axis without employing Pade approximants or non-linear least squares fit and potentially offering superior accuracy. Here, we provide a brief outline of the theory behind CD and give an example of the self-energy calculation within CD without invoking RI in order to facilitate comparison with the results prsented above. 

Detailed discussion of the CD can be found in the following papers:

1. Golze, D., Wilhelm, J., van Setten, M. J., & Rinke, P. (2018). Core-Level Binding Energies from GW : An Efficient Full-Frequency Approach within a Localized Basis. Journal of Chemical Theory and Computation, 14(9), 4856–4869. https://doi.org/10.1021/acs.jctc.8b00458

2. Giantomassi, M., Stankovski, M., Shaltaf, R., Grüning, M., Bruneval, F., Rinke, P., & Rignanese, G.-M. (2011). Electronic properties of interfaces and defects from many-body perturbation theory: Recent developments and applications. Physica Status Solidi (B), 248(2), 275–289. https://doi.org/10.1002/pssb.201046094

CD is used to recast the convolution in the GW expression of self-energy as a difference between two integrals, one which can be performed analytically whereas the other can be evaluated numerically on a relatively small grid. This is achieved by closing the inegration contour as shown below [2]:

![Integration contour used to evaluate $\Sigma(\omega)$](CD_scheme.jpg)

$$
\Sigma(r_1,r_2, \omega) = \frac{i}{2\pi} \int_{-\infty}^{+\infty} e^{i\omega^{\prime} \eta} G(r_1, r_2, \omega + \omega^{\prime}) W(r_1, r_2, \omega^{\prime}) d\omega^{\prime}\\ 
= \frac{i}{2\pi} \oint_{\Gamma} G(r_1, r_2, \omega + z) W(r_1, r_2, z) dz -  \frac{1}{2\pi} \int_{-\infty}^{+\infty} G(r_1, r_2, \omega + i\omega^{\prime}) W(r_1, r_2, i\omega^{\prime}) d\omega^{\prime}
$$

Depending on the $\omega$ value the lower-left and the upper-right loops of the contour can enclose one or several poles of the zero-order Green's function whereas the poles of the screened Coulomb interaction never fall within the contour. This allowes to evaluate the countour integral as a sum of corresponding residues with apropriate signs (note that the upper-right loop is traversed counter-clockwise, while the lower-left loop is traversed clockwise). The imaginary axis contribution is calculated using Gauss-Legendre grid. Importantly, the intgrals over the arches vanish iff the screened Coulomb interaction does not contain the exchange contribution.

In [1]:
import psi4
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container {width:95% !important;}</style>"))


In [3]:

psi4.set_options({'basis' : 'cc-pVDZ', 'd_convergence' : 1e-7,'scf_type' : 'out_of_core', 'dft_spherical_points' : 974, 'dft_radial_points' : 150 })

h2o = psi4.geometry("""O  0.0000 0.0000 0.0000
                       H  0.7571 0.0000 0.5861
                       H -0.7571 0.0000 0.5861
                       symmetry c1
                       units angstrom
""")



psi4.set_output_file('h2o_ccpvdz.out')

scf_e, scf_wfn = psi4.energy('PBE', return_wfn=True)


print("DFT energy is %16.10f" % scf_e)
epsilon = np.asarray(scf_wfn.epsilon_a())
print(epsilon*psi4.constants.hartree2ev)

DFT energy is   -76.3334180928
[-509.90743857  -24.5705732   -12.44826717   -8.26782379   -6.119171
    0.93196052    2.98443149   14.14037229   15.49942683   23.22940234
   23.67842198   25.84136351   31.39893138   32.08395389   37.36815154
   41.8311064    43.94079227   55.70959425   56.71619342   77.1235222
   78.29498123   82.81054474   91.91237823   99.19089852]


``` SCF Total Energy (Ha):      -76.3334178338 (MOLGW) ```
``` SCF/PBE Total Energy (Ha):  -76.333418108953 (NWCHEM) ```


In [4]:
import GW

In [5]:
gw_par = {'no_qp' : 2, 'nv_qp' : 1, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 200, 'debug' : False, 'low_mem' : True, 'analytic_W' : True }
gw_h2o_test = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_test.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (3, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.023 Gb
Calculation of the residue term requires     0.058 Gb
Using low-memory algorithm
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-0.26441299 -0.26875991  0.16223445

In [5]:
gw_par = {'no_qp' : 5, 'nv_qp' : 0, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 200, 'debug' : False, 'low_mem' : True }
gw_h2o_dz_cd1_test = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_cd1_test.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (5, 501)
Caculating GW self-energy via contour deformation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Calculation of the integral term requires    0.023 Gb
Calculation of the residue term requires     0.058 Gb
Using low-memory algorithm
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.23944534 -0.26441299 -0.26875991]
Perfoming graphic solution of the inverse Dyson equation
Done!
E^lin, e

In [6]:
gw_par = {'no_qp' : 5, 'nv_qp' : 19, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 200, 'debug' : False, 'low_mem' : True }
gw_h2o_dz_cd1_test_all = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_cd1_test_all.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (24, 501)
Caculating GW self-energy via contour deformation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Calculation of the integral term requires    0.023 Gb
Calculation of the residue term requires     0.058 Gb
Using low-memory algorithm


KeyboardInterrupt: 

```

 GW eigenvalues (eV)
   #        E0         SigX-Vxc       SigC          Z        E_qp^lin     E_qp^graph
   1   -509.966750   -48.873439    17.227624     0.735425  -533.239868  -531.992207
   2    -24.573136   -11.682417     2.463706     0.000000   -24.573136   -30.975759
   3    -12.448966    -6.515780     0.635819     0.898336   -17.731146   -17.843275
   4     -8.268881    -7.194710     1.360682     0.902578   -13.534550   -13.400892
   5     -6.119281    -7.313383     1.646504     0.907361   -11.261188   -11.169544


```

In [7]:
gw_par = {'no_qp' : 5, 'nv_qp' : 0, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 200, 'debug' : False, 'low_mem' : True, 'analytic_W' : True }
gw_h2o_dz_cd1_test1 = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_cd1_test1.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (5, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.023 Gb
Calculation of the residue term requires     0.058 Gb
Using low-memory algorithm
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.23944534

In [9]:
gw_par = {'no_qp' : 5, 'nv_qp' : 19, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 200, 'debug' : False, 'low_mem' : True, 'analytic_W' : True }
gw_h2o_dz_cd1_test1_all = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_cd1_test1_all.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (24, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.023 Gb
Calculation of the residue term requires     0.058 Gb
Using low-memory algorithm
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.2394453

In [7]:
gw_par = {'no_qp' : 5, 'nv_qp' : 0, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 200, 'debug' : False, 'low_mem' : False, 'analytic_W' : True }
gw_h2o_dz_cd1_test1 = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_cd1_test1.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (5, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.023 Gb
Calculation of the residue term requires     1.394 Gb
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.23944534 -0.26441299 -0.26875991]
Perfoming graphic solution of the inverse Dyson equation
Done!
E^lin, eV  E^graph, eV  Z 
  -533.190591    -531.929352       0.735421
   -24.570573     -30.972735       0.000000
   -17.731886     -17.841995       0.898281
   -13.536447     -13.399890       0.902577
   -11.263837     -11.169323       0.907356
Graphical solver

```
Low-memory
E^lin, eV  E^graph, eV  Z 
  -533.190591    -531.929352       0.735421
   -24.570573     -30.972735       0.000000
   -17.731886     -17.841995       0.898281
   -13.536447     -13.399890       0.902577
   -11.263837     -11.169323       0.907356
   
Simple algorithm
E^lin, eV  E^graph, eV  Z 
  -533.190591    -531.929352       0.735421
   -24.570573     -30.972735       0.000000
   -17.731886     -17.841995       0.898281
   -13.536447     -13.399890       0.902577
   -11.263837     -11.169323       0.907356
```


In [10]:
gw_par = {'no_qp' : 5, 'nv_qp' : 0, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 600, 'debug' : False, 'low_mem' : False, 'analytic_W' : True }
gw_h2o_dz_cd1_test2_ = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_cd1_test2_.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (5, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.070 Gb
Calculation of the residue term requires     1.394 Gb
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.23944534 -0.26441299 -0.26875991]
P

```
600 Gauss-Legendre quadrature points
E^lin, eV  E^graph, eV  Z 
  -533.175015    -531.929352       0.735421
   -24.570573     -30.972735       0.000000
   -17.729061     -17.841995       0.898281
   -13.532374     -13.399890       0.902577
   -11.259761     -11.169323       0.907356
   
200 Gauss-Legendre quadrature points
E^lin, eV  E^graph, eV  Z 
  -533.190591    -531.929352       0.735421
   -24.570573     -30.972735       0.000000
   -17.731886     -17.841995       0.898281
   -13.536447     -13.399890       0.902577
   -11.263837     -11.169323       0.907356
   
Reference(MolGW)
 GW eigenvalues (eV)
   #        E0         SigX-Vxc       SigC          Z        E_qp^lin     E_qp^graph
   1   -509.966750   -48.873439    17.227624     0.735425  -533.239868  -531.992207
   2    -24.573136   -11.682417     2.463706     0.000000   -24.573136   -30.975759
   3    -12.448966    -6.515780     0.635819     0.898336   -17.731146   -17.843275
   4     -8.268881    -7.194710     1.360682     0.902578   -13.534550   -13.400892
   5     -6.119281    -7.313383     1.646504     0.907361   -11.261188   -11.169544
```



```
Analytic vs approximate W (contour deformation algorithm)

Analytic
E^lin, eV  E^graph, eV  Z 
  -533.190591    -531.929352       0.735421
   -24.570573     -30.972735       0.000000
   -17.731886     -17.841995       0.898281
   -13.536447     -13.399890       0.902577
   -11.263837     -11.169323       0.907356
   
Approximate
E^lin, eV  E^graph, eV  Z 
  -533.186256    -531.929350       0.735421
   -24.570573     -30.972733       0.000000
   -17.731306     -17.841995       0.898281
   -13.535498     -13.399891       0.902577
   -11.262817     -11.169322       0.907356
   

MolGW reference
 GW eigenvalues (eV)
   #        E0         SigX-Vxc       SigC          Z        E_qp^lin     E_qp^graph
   1   -509.966750   -48.873439    17.227624     0.735425  -533.239868  -531.992207
   2    -24.573136   -11.682417     2.463706     0.000000   -24.573136   -30.975759
   3    -12.448966    -6.515780     0.635819     0.898336   -17.731146   -17.843275
   4     -8.268881    -7.194710     1.360682     0.902578   -13.534550   -13.400892
   5     -6.119281    -7.313383     1.646504     0.907361   -11.261188   -11.169544
```

In [5]:
gw_par = {'no_qp' : 5, 'nv_qp' : 19, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 600, 'debug' : False, 'low_mem' : False, 'analytic_W' : True, 
         'evgw_iter' : 2}
gw_h2o_dz_evgw = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_evgw.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (24, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.070 Gb
Calculation of the residue term requires     1.394 Gb
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.23944534 -0.26441299 -0.26875991  

In [6]:
gw_par = {'no_qp' : 5, 'nv_qp' : 19, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 600, 'debug' : False, 'low_mem' : False, 'analytic_W' : True, 
         'evgw_iter' : 0}
gw_h2o_dz_normal = GW.GW_DFT(scf_wfn, h2o, gw_par)
gw_h2o_dz_normal.print_summary()

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (24, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
eps :  [-18.73875704  -0.90295212  -0.45746549  -0.30383699  -0.22487544
   0.03424893   0.10967586   0.51964922   0.56959356   0.85366499
   0.87016616   0.94965281   1.15388971   1.17906383   1.37325456
   1.53726516   1.6147947    2.0472903    2.08428214   2.83423782
   2.87728815   3.04323209   3.37771837   3.64519912]
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.070 Gb
Calculation of the residue term requires     1.394 Gb
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.23944534 -0.26441299 -0.26875991  

In [9]:
# External version of evGW
gw_par = {'no_qp' : 5, 'nv_qp' : 19, 'nomega_sigma' : 501, 'step_sigma' : 0.01, 'gl_npoint' : 600, 'debug' : False, 'low_mem' : False, 'analytic_W' : True, 
         'evgw_iter' : 0}
e = np.asarray(scf_wfn.epsilon_a())
evgw_iterations = 5
for _ in range(evgw_iterations):
    tmp = GW.GW_DFT(scf_wfn, h2o, gw_par)
    print("Solution :", tmp.qp_molgw_graph_)
    e[:] =  np.copy(tmp.qp_molgw_graph_)
    
    

Number of basis functions:  24
occ/virt: 5/19
Attempting to create RI basis set for CC-PVDZ (RIFIT)... 
Auxiliary basis set has been generated!
Number of auxiliary basis functions:  84
Fraction of HF exchange is  0.000
Running in production mode!
Shape of the omega_grid_all is  (24, 501)
Caculating GW self-energy via contour deformation
Analytic W has been requested; performing RPA calculation
eps :  [-27.62513594  -3.35731976  -2.18899706  -2.05901927  -1.96102506
   1.20478781   1.27479002   2.30261397   2.42684425   2.94278657
   3.1168331    2.90568871   3.1440103    3.27369339   3.46417449
   3.68596919   3.74441635   4.78285355   4.69521012   5.71764676
   6.01127962   6.37462299   6.42493865   6.89031648]
Shape of omega tensor is  (24, 24, 95)
Calculation of the integral term requires    0.070 Gb
Calculation of the residue term requires     1.394 Gb
Finished calculating self-energy
Performing one-shot G0W0
SigX - Vxc
[-1.79604982 -0.42930912 -0.23944534 -0.26441299 -0.26875991  

In [10]:
print(e*psi4.constants.hartree2ev)

[-951.55620933 -141.77787911  -90.80043793  -88.89109944  -86.27686391
   53.62473846   55.53094039   95.90062     101.14057356  118.82895274
  126.03114125  116.34107768  122.61204346  127.31769861  133.0396701
  141.24315356  142.83591001  177.80677312  175.82018827  209.77423118
  218.48309442  229.58966086  232.80576998  245.68801332]
